In [1]:
from pathlib import Path
import sys  
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import numpy as np
from operator import itemgetter

In [1]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

ModuleNotFoundError: No module named 'Utils'

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 146 ( 0.4%) Users that have less than 1 test interactions


In [24]:
np.linspace(0, 2, 20)

array([0.        , 0.10526316, 0.21052632, 0.31578947, 0.42105263,
       0.52631579, 0.63157895, 0.73684211, 0.84210526, 0.94736842,
       1.05263158, 1.15789474, 1.26315789, 1.36842105, 1.47368421,
       1.57894737, 1.68421053, 1.78947368, 1.89473684, 2.        ])

In [13]:
#from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

In [6]:
#data_reader = Movielens10MReader()
#data_loaded = data_reader.load_data()

#URM_all = data_loaded.get_URM_all()

In [17]:
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [22]:
URM_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8000043 stored elements and shape (69878, 10681)>

In [ ]:
#MAP = []
URM_train_coo = URM_train.tocoo()


for topK_ in [50, 100, 150, 200]:
    for l1_ratio_ in np.linspace(0.001, 1, 10):
        for alpha_ in np.linspace(0.00001, 0.0005, 3):
            slim_model = SLIMElasticNetRecommender(URM_train_coo, verbose=True)
            slim_model.fit(l1_ratio=l1_ratio_, alpha = alpha_, topK = topK_)

            result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)
            MAP.append(('topK={}'.format(topK_), 'l1_ratio={}'.format(l1_ratio_), 'alpha={}'.format(alpha_), result_dict["MAP"][10]))

            print(MAP[-1])
            

SLIMElasticNetRecommender: Processed 5112 (13.4%) in 5.00 min. Items per second: 17.04
SLIMElasticNetRecommender: Processed 10228 (26.8%) in 10.00 min. Items per second: 17.04
SLIMElasticNetRecommender: Processed 15351 (40.3%) in 15.00 min. Items per second: 17.05
SLIMElasticNetRecommender: Processed 20487 (53.7%) in 20.00 min. Items per second: 17.07
SLIMElasticNetRecommender: Processed 25527 (67.0%) in 25.00 min. Items per second: 17.02
SLIMElasticNetRecommender: Processed 30553 (80.1%) in 30.00 min. Items per second: 16.97
SLIMElasticNetRecommender: Processed 35782 (93.9%) in 35.00 min. Items per second: 17.04
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 37.39 min. Items per second: 16.99
EvaluatorHoldout: Processed 35572 (100.0%) in 9.80 sec. Users per second: 3632
('topK=50', 'l1_ratio=0.001', 'alpha=1e-05', 0.056627780633597836)
SLIMElasticNetRecommender: Processed 8016 (21.0%) in 5.00 min. Items per second: 26.71
SLIMElasticNetRecommender: Processed 16162 (42.4%) in 10

In [14]:
max_tuple = max(MAP, key=itemgetter(3))
print('The best combination for the ICF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

The best combination for the ICF recommender is (topK=20,l1_ratio=0.15873684210526315,alpha=0.00034), with a MAP = 0.05992418684929483


In [ ]:
slim_model = SLIMElasticNetRecommender(URM_train)

In [14]:
slim_model.fit()

SLIMElasticNetRecommender: Processed 38121 (100.0%) in 3.09 min. Items per second: 205.70


In [16]:
result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)

EvaluatorHoldout: Processed 35590 (100.0%) in 8.42 sec. Users per second: 4225


In [17]:
result_dict

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000865,0.001183,0.000963,0.000252,0.000365,0.002385,0.001031,0.000912,0.008205,0.00245,...,0.995914,0.008171,0.995914,0.000638,4.95516,0.955743,0.00105,0.335671,2.365443,0.232804


In [18]:
result_dict["MAP"][10]

0.00025152753805465186

# SLIM BPR

In [7]:
from libs.Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

In [8]:
MAP_bpr = []

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [10]:
URM_train_coo = URM_train.tocoo()


for topK_ in [200, 300, 400]:
    for learning_rate_ in np.linspace(0.001, 1, 4):
        for lambda_i_ in np.linspace(0.005, 0.01, 4):
            for lambda_j_ in np.linspace(0.005, 0.01, 4):
                slim_bpr_model = SLIM_BPR_Python(URM_train_coo, verbose=False)
                slim_bpr_model.fit(learning_rate=learning_rate_,
                                   lambda_i=lambda_i_,
                                   lambda_j=lambda_j_,
                                   topK = topK_,
                                   epochs=100,
                                  )
    
                result_dict, _ = evaluator_validation.evaluateRecommender(slim_bpr_model)
                MAP_bpr.append(('topK={}'.format(topK_),
                                'learning_rate={}'.format(learning_rate_),
                                'lambda_i_={}'.format(lambda_i_),
                                'lambda_j_={}'.format(lambda_j_),
                                result_dict["MAP"][10]))
    
                print(MAP_bpr[-1])
            

Train completed in 3.08 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 17.81 sec. Users per second: 1998
('topK=200', 'learning_rate=0.001', 'lambda_i_=0.005', 'lambda_j_=0.005', 0.030668128126519298)
Train completed in 3.35 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 17.78 sec. Users per second: 2000
('topK=200', 'learning_rate=0.001', 'lambda_i_=0.005', 'lambda_j_=0.006666666666666667', 0.030627258569062735)
Train completed in 3.33 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 17.57 sec. Users per second: 2024
('topK=200', 'learning_rate=0.001', 'lambda_i_=0.005', 'lambda_j_=0.008333333333333333', 0.030665032285990212)
Train completed in 3.08 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 18.27 sec. Users per second: 1947
('topK=200', 'learning_rate=0.001', 'lambda_i_=0.005', 'lambda_j_=0.01', 0.03056500930425509)
Train completed in 3.12 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 17.87 sec. Users per second: 1990
('topK=200', 'learning_rate=

/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/SLIM/SLIM_BPR_Python.py:78: RuntimeWarning: overflow encountered in exp
  sigmoid_gradient = 1 / (1 + np.exp(x_uij))


Train completed in 3.70 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 15.97 sec. Users per second: 2227
('topK=200', 'learning_rate=0.334', 'lambda_i_=0.005', 'lambda_j_=0.008333333333333333', 0.035371372567393256)


/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/SLIM/SLIM_BPR_Python.py:78: RuntimeWarning: overflow encountered in exp
  sigmoid_gradient = 1 / (1 + np.exp(x_uij))


Train completed in 3.64 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 16.60 sec. Users per second: 2143
('topK=200', 'learning_rate=0.334', 'lambda_i_=0.005', 'lambda_j_=0.01', 0.035436725482057396)


...

Train completed in 4.30 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 16.05 sec. Users per second: 2216
('topK=200', 'learning_rate=1.0', 'lambda_i_=0.005', 'lambda_j_=0.005', 0.02698147850649889)


/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/SLIM/SLIM_BPR_Python.py:78: RuntimeWarning: overflow encountered in exp
  sigmoid_gradient = 1 / (1 + np.exp(x_uij))


Train completed in 4.37 minutes
EvaluatorHoldout: Processed 35570 (100.0%) in 16.10 sec. Users per second: 2209
('topK=200', 'learning_rate=1.0', 'lambda_i_=0.005', 'lambda_j_=0.006666666666666667', 0.026692474262687265)


/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/SLIM/SLIM_BPR_Python.py:78: RuntimeWarning: overflow encountered in exp
  sigmoid_gradient = 1 / (1 + np.exp(x_uij))


KeyboardInterrupt: 